In [1]:
from maxpooling import MaxPool
import numpy as np

def test_maxpool_forward():
    # Use a fixed input with distinct values for clarity.
    x = np.array([[[[0, 1, 2, 3],
                    [4, 5, 6, 7],
                    [8, 9, 10, 11],
                    [12, 13, 14, 15]]]], dtype=np.float32)
    # Expected output for kernel_size=2, stride=2, padding=0:
    # For each 2x2 window, the max values are:
    # Window 1: [[0,1],[4,5]] -> max is 5
    # Window 2: [[2,3],[6,7]] -> max is 7
    # Window 3: [[8,9],[12,13]] -> max is 13
    # Window 4: [[10,11],[14,15]] -> max is 15
    expected_out = np.array([[[[5, 7],
                               [13, 15]]]], dtype=np.float32)

    pool = MaxPool(kernel_size=2, stride=2, padding=0)
    out = pool.forward(x)
    
    print("Forward Pass Test:")
    print("Input:\n", x)
    print("Expected Output:\n", expected_out)
    print("MaxPool Forward Output:\n", out)
    
    assert np.allclose(out, expected_out), "Forward pass output does not match expected values."
    print("Forward pass test passed.\n")

def test_maxpool_backward():
    # Use the same fixed input.
    x = np.array([[[[0, 1, 2, 3],
                    [4, 5, 6, 7],
                    [8, 9, 10, 11],
                    [12, 13, 14, 15]]]], dtype=np.float32)
    pool = MaxPool(kernel_size=2, stride=2, padding=0)
    out = pool.forward(x)
    
    # Assume the loss is simply the sum of the outputs,
    # so the upstream gradient is an array of ones.
    d_out = np.ones_like(out)
    d_x = pool.backward(d_out)
    
    # For max pooling, the gradient with respect to the input is 1 at the location
    # of the max in each pooling window, and 0 elsewhere.
    expected_d_x = np.zeros_like(x)
    # Window 1 (top-left 2x2): max at position (1,1)
    expected_d_x[0, 0, 1, 1] = 1.
    # Window 2 (top-right 2x2): max at position (1,3)
    expected_d_x[0, 0, 1, 3] = 1.
    # Window 3 (bottom-left 2x2): max at position (3,1)
    expected_d_x[0, 0, 3, 1] = 1.
    # Window 4 (bottom-right 2x2): max at position (3,3)
    expected_d_x[0, 0, 3, 3] = 1.
    
    print("Backward Pass Test (Analytical Gradients):")
    print("Input:\n", x)
    print("Upstream Gradient d_out:\n", d_out)
    print("Expected d_x:\n", expected_d_x)
    print("Computed d_x:\n", d_x)
    
    assert np.allclose(d_x, expected_d_x), "Backward pass gradients do not match expected values."
    print("Backward pass test (analytical) passed.\n")

def test_maxpool_backward_numerical():
    """
    Numerical gradient check for the maxpool layer with respect to its input.
    The function being tested is f(x) = sum(maxpool(x)),
    so dL/dx is approximated using central differences.
    """
    np.random.seed(42)
    # Use a small input to avoid boundary issues.
    x = np.random.randn(1, 1, 4, 4).astype(np.float64)
    pool = MaxPool(kernel_size=2, stride=2, padding=0)
    
    # Define a function that returns the sum of the maxpool output.
    def f(x_input):
        # Each call to forward caches variables in the pool object.
        return np.sum(pool.forward(x_input))
    
    # Compute the analytical gradient.
    out = pool.forward(x)
    d_out = np.ones_like(out)
    d_x_analytical = pool.backward(d_out)
    
    # Numerical gradient: use a small epsilon for central differences.
    epsilon = 1e-5
    d_x_numerical = np.zeros_like(x)
    
    # Iterate over every element in x.
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        original_value = x[idx]
        
        # Compute f(x + epsilon)
        x[idx] = original_value + epsilon
        f_plus = f(x)
        
        # Compute f(x - epsilon)
        x[idx] = original_value - epsilon
        f_minus = f(x)
        
        # Restore the original value.
        x[idx] = original_value
        
        # Compute the numerical gradient.
        d_x_numerical[idx] = (f_plus - f_minus) / (2 * epsilon)
        it.iternext()
    
    print("Backward Pass Numerical Gradient Check:")
    print("Analytical d_x:\n", d_x_analytical)
    print("Numerical d_x:\n", d_x_numerical)
    
    # Allow a small tolerance, especially since max pooling is non-smooth.
    assert np.allclose(d_x_analytical, d_x_numerical, atol=1e-5), "Numerical gradient check failed."
    print("Numerical gradient check passed.\n")

if __name__ == '__main__':
    test_maxpool_forward()
    test_maxpool_backward()
    test_maxpool_backward_numerical()

Forward Pass Test:
Input:
 [[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]]]
Expected Output:
 [[[[ 5.  7.]
   [13. 15.]]]]
MaxPool Forward Output:
 [[[[ 5.  7.]
   [13. 15.]]]]
Forward pass test passed.

Backward Pass Test (Analytical Gradients):
Input:
 [[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]]]
Upstream Gradient d_out:
 [[[[1. 1.]
   [1. 1.]]]]
Expected d_x:
 [[[[0. 0. 0. 0.]
   [0. 1. 0. 1.]
   [0. 0. 0. 0.]
   [0. 1. 0. 1.]]]]
Computed d_x:
 [[[[0. 0. 0. 0.]
   [0. 1. 0. 1.]
   [0. 0. 0. 0.]
   [0. 1. 0. 1.]]]]
Backward pass test (analytical) passed.

Backward Pass Numerical Gradient Check:
Analytical d_x:
 [[[[1. 0. 0. 0.]
   [0. 0. 1. 0.]
   [0. 1. 1. 0.]
   [0. 0. 0. 0.]]]]
Numerical d_x:
 [[[[1. 0. 0. 0.]
   [0. 0. 1. 0.]
   [0. 1. 1. 0.]
   [0. 0. 0. 0.]]]]
Numerical gradient check passed.

